## Тема 1.6. Сбор данных. Лекция: Сбор данных в Internet

---

### 1. Введение

Сбор данных в интернете (web scraping, веб-скрапинг) — это процесс автоматизированного получения информации с веб-сайтов. Эта технология широко используется для создания агрегаторов, аналитики рынка, мониторинга цен, создания информационных баз и во многих других задачах.

### 2. Основные методы сбора данных в интернете

1. **Загрузка файлов напрямую по ссылке** (например, CSV, XLS, TXT, HTML).
2. **Работа с API** — если сайт предоставляет программный интерфейс (API), лучше использовать его.
3. **Веб-скрапинг** (извлечение данных непосредственно с HTML-страниц).

**ВНИМАНИЕ:** Всегда изучайте условия использования сайта! Некоторые сайты запрещают автоматизированный сбор данных.

---

### 3. Базовые инструменты на Python

- **requests** — для скачивания веб-страниц.
- **BeautifulSoup** — для парсинга HTML-кода.
- **pandas** — для хранения и обработки собранных данных.
- **re (регулярные выражения)** — полезны для извлечения структурированных данных из текста.

#### Установка зависимостей

```bash
pip install requests beautifulsoup4 pandas
```

---

### 4. Сбор HTML-страницы с помощью requests

```python
import requests

url = 'https://example.com'
response = requests.get(url)

print(f'Код ответа: {response.status_code}')  # 200 - OK

print(response.text[:500])  # первые 500 символов HTML
```

---

### 5. Парсинг HTML с помощью BeautifulSoup

```python
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')

# Получение заголовка страницы
title = soup.title.text
print(f'Заголовок: {title}')

# Получение всех ссылок с сайта
links = [a['href'] for a in soup.find_all('a', href=True)]
print(links)
```

---

### 6. Пример: Парсинг таблицы с сайта

Допустим, на странице есть HTML-таблица с данными. Вот пример кода для её извлечения:

```python
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://translated.turbopages.org/proxy_u/en-ru.ru.7d534d03-68f763f4-f9cd49da-74722d776562/https/en.wikipedia.org/wiki/Table_(information)'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

table = soup.find('table')

# Считываем строки таблицы
rows = []
for tr in table.find_all('tr'):
    cols = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
    if cols:
        rows.append(cols)

# Создаём DataFrame из таблицы
df = pd.DataFrame(rows[1:], columns=rows[0])
print(df)
```

---

### 7. Работа с API для сбора данных

Когда сайт предоставляет API, сбор данных проще и юридически корректнее:

```python
import requests

CITY = "Moscow"
API_KEY = "c86250163953d321b9b61edeb46ad4c7"
url = f"http://api.openweathermap.org/data/2.5/weather"
params = {
    'q': CITY,
    'appid': API_KEY,
    'units': 'metric',  # для температуры в градусах Цельсия
    'lang': 'ru'        # для русского языка
}

response = requests.get(url, params=params)
print(response.json())
```

Преобразование в табличный вид

```python
cities = ['Moscow', 'London', 'Paris', 'Vladivostok', 'Barselona']
rows = []

for city in cities:
    params['q'] = city
    r = requests.get(url, params=params)
    d = r.json()
    row = {
        'city': d['name'],
        'datetime': pd.to_datetime(d['dt'], unit='s'),
        'temp': d['main']['temp'],
        'humidity': d['main']['humidity'],
        'pressure': d['main']['pressure'],
        'wind_speed': d['wind']['speed'],
        'weather': d['weather'][0]['description']
    }
    rows.append(row)

df = pd.DataFrame(rows)
df
```

---

### 8. Примеры ошибок и обработка

```python
import time

for page in range(1, 6):
    try:
        response = requests.get(f'https://example.com/page={page}', timeout=5)
        print(response.status_code)
        time.sleep(1)
    except requests.exceptions.RequestException as e:
        print(f'Ошибка: {e}')
```



## Задания
### 1. Получите HTML-код главной страницы Википедии (`https://ru.wikipedia.org/wiki/Главная_страница`) и выведите первые 300 символов.
**Ответ:**
```python
import requests
r = requests.get('https://ru.wikipedia.org/wiki/Главная_страница')
print(r.text[:300])
```

---

### 2. Выведите статус ответа сайта https://www.python.org.
**Ответ:**
```python
print(requests.get('https://www.python.org').status_code)
```

---

### 3. Найдите текст заголовка (title) главной страницы GitHub.
**Ответ:**
```python
from bs4 import BeautifulSoup
r = requests.get('https://github.com')
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.title.text)
```

---

### 4. Получите список всех ссылок (href) на странице https://www.bbc.com/news.
**Ответ:**
```python
from bs4 import BeautifulSoup
r = requests.get('https://www.bbc.com/news')
soup = BeautifulSoup(r.text, 'html.parser')
links = [a['href'] for a in soup.find_all('a', href=True)]
print(links)
```

---

### 5. Определите, есть ли на https://stackoverflow.com тег h1 и если да — выведите его текст.
**Ответ:**
```python
soup = BeautifulSoup(requests.get('https://stackoverflow.com').text, 'html.parser')
h1 = soup.find('h1')
if h1: print(h1.text)
```

---

### 6. Найдите все изображения на https://www.imdb.com/chart/top и выведите 5 первых их абсолютных ссылок.
**Ответ:**
```python
r = requests.get('https://www.imdb.com/chart/top')
soup = BeautifulSoup(r.text, 'html.parser')
imgs = [img['src'] for img in soup.find_all('img', src=True)]
print(imgs[:5])  # Возможно, придется обработать относительные пути.
```

---

### 7. Извлеките все подзаголовки (h3) главной страницы https://www.bbc.com и выведите 10 первых.
**Ответ:**
```python
r = requests.get('https://www.bbc.com')
soup = BeautifulSoup(r.text, 'html.parser')
print([h.text.strip() for h in soup.find_all('h3')][:10])
```

### 8. Подключитесь к API hh.ru https://api.hh.ru/vacancies
**Ответ:**  
```python
import requests
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

base_url = 'https://api.hh.ru/vacancies'
params = {
    'per_page': 100,
    'page': 0
}

vacancies = []
pages_number = 15

for page in range(pages_number):
    params['page'] = page
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        page_data = response.json()
        vacancies.extend(page_data['items'])
    else:
        print(f"Не удалось получить данные с страницы {page}")
        break

df = pd.DataFrame(vacancies)
df['region'] = df['area'].apply(lambda x: x['name'])
```